<a href="https://colab.research.google.com/github/DavoII/handwritten_letters_recognition/blob/master/Armenian_Letters(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [6]:
data = np.load('drive/My Drive/uppercase_without_Զ/data.npy', encoding='bytes') 
labels = np.load('drive/My Drive/uppercase_without_Զ/labels.npy', encoding='bytes') 
class_idx = np.load('drive/My Drive/uppercase_without_Զ/label_classes.npy', encoding='bytes')

print(data.shape)
print(labels.shape)

(40845, 100, 100)
(40845, 38)


In [7]:
number_classes = len(class_idx)
print('number of classes', number_classes)

number of classes 38


In [0]:
x_train, x_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=labels
    )

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_train /= 255
x_test /= 255

In [0]:
(img_shape_w, img_shape_h) = x_train[0].shape
x_train = x_train.reshape(x_train.shape[0], img_shape_w, img_shape_h, 1)
x_test = x_test.reshape(x_test.shape[0], img_shape_w, img_shape_h, 1)

In [12]:
print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_train (28591, 100, 100, 1)
y_train (28591, 38)
x_test (12254, 100, 100, 1)
y_test (12254, 38)


In [0]:
# model fit with generator
'''
gen = ImageDataGenerator(rotation_range=12, width_shift_range=0.1, shear_range=0.3,
                        height_shift_range=0.1, zoom_range=0.1, data_format='channels_first')

batch_size = 512
batches = gen.flow(x_train, y_train, batch_size=batch_size)
test_batches = gen.flow(x_test, y_test, batch_size=batch_size)
steps_per_epoch = int(np.ceil(batches.n/batch_size))
validation_steps = int(np.ceil(test_batches.n/batch_size))

model.fit(
    batches,
    epochs=4,
    validation_data=test_batches,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
    )
'''

In [0]:
def build_model(hp):

  categories=number_classes

  ly1_neurons = hp.Int('conv_1_filter', min_value=128, max_value=512, step=16)
  ly2_neurons = hp.Int('conv_2_filter', min_value=128, max_value=512, step=16)
  ly3_neurons = hp.Int('conv_3_filter', min_value=128, max_value=512, step=16)

  kernel_size_1 = hp.Choice('conv_1_kernel', values=[3, 5, 7])
  kernel_size_2 = hp.Choice('conv_2_kernel', values=[3, 5, 7])
  kernel_size_3 = hp.Choice('conv_3_kernel', values=[3, 5, 7])

  ly1_dropout = hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.25, step=0.05)
  ly2_dropout = hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)
  ly3_dropout = hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.25, step=0.05)

  pool_size_1 = (2, 2)
  pool_size_2 = (2, 2)
  pool_size_3 = (2, 2)

  model = Sequential()
  
  model.add(Conv2D(filters=ly1_neurons, kernel_size=kernel_size_1, input_shape=x_train.shape[1:]))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=pool_size_1))
  model.add(Dropout(ly1_dropout))

  model.add(Conv2D(filters=ly2_neurons, kernel_size=kernel_size_2))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=pool_size_2))
  model.add(Dropout(ly2_dropout))

  model.add(Conv2D(filters=ly3_neurons, kernel_size=kernel_size_3))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=pool_size_3))
  model.add(Dropout(ly3_dropout))

  model.add(Flatten())
  model.add(Dense( 512 ))
  model.add(Activation('relu'))
  model.add(Dense( categories ))
  model.add(Activation('softmax'))

  model.compile(
      optimizer=optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-4])),
      loss='categorical_crossentropy',
      metrics=['accuracy']
      )

  return model


In [0]:
!pip install keras-tuner

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [30]:
tuner_search = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    )

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [0]:
tuner_search.search(x_train, y_train, epochs=5, validation_split=0.2)

In [0]:
model_from_tunner = tuner_search.get_best_models(num_models=1)[0]

In [33]:
model_from_tunner.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 176)       4576      
_________________________________________________________________
activation (Activation)      (None, 96, 96, 176)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 176)       0         
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 176)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 496)       2182896   
_________________________________________________________________
activation_1 (Activation)    (None, 44, 44, 496)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 496)       0

In [34]:
model_from_tunner.fit(
    x_train,
    y_train,
    batch_size=120,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
    )

Epoch 1/10
239/239 [==============================] - 147s 615ms/step - loss: 0.1962 - accuracy: 0.9408 - val_loss: 0.2483 - val_accuracy: 0.9292
Epoch 2/10
239/239 [==============================] - 145s 605ms/step - loss: 0.1696 - accuracy: 0.9488 - val_loss: 0.2433 - val_accuracy: 0.9297
Epoch 3/10
239/239 [==============================] - 145s 605ms/step - loss: 0.1507 - accuracy: 0.9532 - val_loss: 0.2332 - val_accuracy: 0.9334
Epoch 4/10
239/239 [==============================] - 145s 605ms/step - loss: 0.1367 - accuracy: 0.9569 - val_loss: 0.2365 - val_accuracy: 0.9324
Epoch 5/10
239/239 [==============================] - 145s 605ms/step - loss: 0.1221 - accuracy: 0.9610 - val_loss: 0.2423 - val_accuracy: 0.9323
Epoch 6/10
239/239 [==============================] - 145s 605ms/step - loss: 0.1151 - accuracy: 0.9623 - val_loss: 0.2412 - val_accuracy: 0.9326
Epoch 7/10
239/239 [==============================] - 145s 605ms/step - loss: 0.1050 - accuracy: 0.9651 - val_loss: 0.2444 -